In [1]:
!pip install datasets

In [2]:
from datasets import load_dataset

# Load the dataset
dataset = load_dataset("SKNahin/bengali-transliteration-data")

# Split into train and validation sets (80/20 split)
dataset = dataset["train"].train_test_split(test_size=0.2, seed=42)

train_dataset = dataset["train"]
val_dataset = dataset["test"]

print(f"Training samples: {len(train_dataset)}, Validation samples: {len(val_dataset)}")

README.md:   0%|          | 0.00/300 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/333k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/5006 [00:00<?, ? examples/s]

Training samples: 4004, Validation samples: 1002


In [3]:
from transformers import AutoTokenizer

# Initialize tokenizer (we'll select the model in Task 3)
tokenizer = AutoTokenizer.from_pretrained("t5-small")  # Example model



The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

In [4]:
def preprocess_function(examples):
    inputs = examples["rm"]  # Banglish (Romanized Bengali) text
    targets = examples["bn"]  # Bengali script text
    model_inputs = tokenizer(inputs, max_length=128, truncation=True, padding="max_length")
    labels = tokenizer(targets, max_length=128, truncation=True, padding="max_length").input_ids
    model_inputs["labels"] = labels
    return model_inputs


In [5]:
# Apply the preprocessing function to the datasets
train_dataset = train_dataset.map(preprocess_function, batched=True)
val_dataset = val_dataset.map(preprocess_function, batched=True)

# Set format for PyTorch
train_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])
val_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

Map:   0%|          | 0/4004 [00:00<?, ? examples/s]

Map:   0%|          | 0/1002 [00:00<?, ? examples/s]

In [6]:
from transformers import AutoModelForSeq2SeqLM

# Load pre-trained mT5 model
model = AutoModelForSeq2SeqLM.from_pretrained("google/mt5-small")

config.json:   0%|          | 0.00/553 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [7]:
from transformers import Trainer, TrainingArguments

In [11]:
import torch
from torch.utils.data import DataLoader
from transformers import AdamW
from tqdm import tqdm


In [13]:
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=8)

# Define optimizer
optimizer = AdamW(model.parameters(), lr=5e-5)

# Set up training parameters
num_epochs = 3
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Training loop
for epoch in range(num_epochs):
    model.train()
    train_loss = 0

    # Training phase
    loop = tqdm(train_loader, leave=True)
    for batch in loop:
        # Move data to the device
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        # Forward pass
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss

        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        train_loss += loss.item()

        # Update progress bar
        loop.set_description(f"Epoch {epoch + 1}")
        loop.set_postfix(loss=loss.item())

    avg_train_loss = train_loss / len(train_loader)
    print(f"Training Loss for Epoch {epoch + 1}: {avg_train_loss:.4f}")

    # Validation phase
    model.eval()
    val_loss = 0

    with torch.no_grad():
        for batch in val_loader:
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device)

            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            val_loss += outputs.loss.item()

    avg_val_loss = val_loss / len(val_loader)
    print(f"Validation Loss for Epoch {epoch + 1}: {avg_val_loss:.4f}")

Epoch 1: 100%|██████████| 501/501 [04:15<00:00,  1.96it/s, loss=6.12]


Training Loss for Epoch 1: 22.4071
Validation Loss for Epoch 1: 7.8087


Epoch 2: 100%|██████████| 501/501 [04:20<00:00,  1.92it/s, loss=1.99]


Training Loss for Epoch 2: 3.7735
Validation Loss for Epoch 2: 1.0182


Epoch 3: 100%|██████████| 501/501 [04:20<00:00,  1.92it/s, loss=1.36] 


Training Loss for Epoch 3: 1.3876
Validation Loss for Epoch 3: 0.4040


In [15]:
# Ensure tensors are contiguous
for param in model.parameters():
    if not param.is_contiguous():
        param.data = param.data.contiguous()

# Save the model and tokenizer
model.save_pretrained("./fine_tuned_model")
tokenizer.save_pretrained("./fine_tuned_model")

print("Model and tokenizer saved successfully!")

Model and tokenizer saved successfully!


In [26]:
!pip install huggingface_hub


In [29]:
from kaggle_secrets import UserSecretsClient

user_secrets = UserSecretsClient()
huggingface_token = user_secrets.get_secret("HUGGINGFACE_TOKEN")
print("Token retrieved successfully")

Token retrieved successfully


In [30]:
from huggingface_hub import login

# Log in to Hugging Face using the token
login(token=huggingface_token)


The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [31]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

# Save your model and tokenizer
model_path = "./fine_tuned_model"
model.save_pretrained(model_path)
tokenizer.save_pretrained(model_path)


('./fine_tuned_model/tokenizer_config.json',
 './fine_tuned_model/special_tokens_map.json',
 './fine_tuned_model/spiece.model',
 './fine_tuned_model/added_tokens.json',
 './fine_tuned_model/tokenizer.json')

In [32]:
from huggingface_hub import HfApi

# Set your repository name (e.g., your-username/your-model-name)
repo_name = "namisa/banglish-to-bangla-model"

# Create the repository (set private=True for private repos)
api = HfApi()
repo_url = api.create_repo(repo_id=repo_name, private=False)
print(f"Repository created: {repo_url}")


Repository created: https://huggingface.co/namisa/banglish-to-bangla-model


In [33]:
from huggingface_hub import upload_folder

# Upload the saved model to the Hugging Face repository
upload_folder(
    repo_id=repo_name,
    folder_path=model_path,
    commit_message="Uploading fine-tuned model from Kaggle"
)

print(f"Model uploaded successfully to {repo_url}")


spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.20G [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

Model uploaded successfully to https://huggingface.co/namisa/banglish-to-bangla-model
